In [42]:
import xml.etree.ElementTree as ET
import pandas as pd
#with open('./arXiv_src_manifest.xml', 'r') as f:
#    mani = ET.parse(f)

In [15]:
root = mani.getroot()
def parse_element(elem):
    return_dict = {}
    for e in elem:
        return_dict[e.tag] = e.text
    return return_dict
def parse_root(root):
        return [parse_element(child) for child in iter(root) if child.tag != 'timestamp']

filedf = pd.DataFrame(parse_root(root))
filedf[['num_items', 'size']] = filedf[['num_items', 'size']].astype(int)
filedf['filename'] = filedf['filename'].astype(str)

In [16]:
import hashlib 

In [71]:
hash_md5 = hashlib.md5()
with open('../arXiv_src_1804_001.tar', "rb") as f:
    for chunk in iter(lambda: f.read(4096), b""):
        hash_md5.update(chunk)
hash_md5.hexdigest()

'fe34df50d3426c745ea9a8b5ed924859'

In [72]:
filedf.loc[filedf['filename'].str.contains('1804_001')]

,content_md5sum,filename,first_item,last_item,md5sum,num_items,seq_num,size,timestamp,yymm
1742,d2ca6595ddb8055262ea1c6adf4cd4e3,src/arXiv_src_1804_001.tar,1804.00001,1804.00262,fe34df50d3426c745ea9a8b5ed924859,262,1,526207253,2018-06-05 04:56:42,1804


In [7]:
S = filedf['size']

In [9]:
S.sum()

924382365733

In [46]:
import subprocess
P = subprocess.run(['/bin/s3cmd', 'get', '--requester-pays', 's3://arxiv/src/arXiv_src_1804_006.tar', '..'], 
                   stderr=subprocess.PIPE,
                   stdout=subprocess.PIPE)

In [41]:
P.stdout

b"download: 's3://arxiv/src/arXiv_src_1804_006sab.tar' -> '../arXiv_src_1804_006sab.tar'  [1 of 1]\ndownload: 's3://arxiv/src/arXiv_src_1804_006sab.tar' -> '../arXiv_src_1804_006sab.tar'  [1 of 1]\n"

In [42]:
P.stderr

b'ERROR: S3 error: 404 (NoSuchKey): The specified key does not exist.\n'

In [45]:
P.check_returncode()

CalledProcessError: Command '['/bin/s3cmd', 'get', '--requester-pays', 's3://arxiv/src/arXiv_src_1804_006sab.tar', '..']' returned non-zero exit status 12.

In [19]:
df = filedf.head()
notdf = pd.DataFrame([{'filename': 'src/arXiv_src_0001_001.tar'}])

In [74]:
idx = df.filename.isin(notdf.filename)
df[~idx].iloc[0]

content_md5sum    d90df481661ccdd7e8be883796539743
filename                src/arXiv_src_0002_001.tar
first_item                         astro-ph0002001
last_item                          quant-ph0002094
md5sum            4592ab506cf775afecf4ad560d982a00
num_items                                     2365
seq_num                                          1
size                                     227036528
timestamp                      2010-12-23 00:18:09
yymm                                          0002
Name: 1, dtype: object

In [55]:
df.filename.isin(notdf.filename)

0    False
1     True
2    False
3    False
4    False
Name: filename, dtype: bool

In [57]:
df.to_csv('head_df.csv')

In [58]:
notdf.to_csv('notdf.csv')

In [35]:
nondf = pd.read_csv('/home/luis/mountpoint/arXiv_src/downloaded_log.csv',index_col=0)
nondf.append(pd.DataFrame([{'filename':'hola'}]), ignore_index=True)

/usr/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Unnamed: 0.1,filename
0,NaN,hola


In [88]:
import os
os.path.ismount('/mnt')

True

In [39]:
notdf

,filename
0,src/arXiv_src_0001_001.tar


In [69]:
#filedf.to_csv('/home/luis/mountpoint/allfiles.csv')
# CAReFulll notdf.to_csv('/home/luis/mountpoint/arXiv_src/downloaded_log.csv')

NameError: name 'notdf' is not defined

In [176]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
from dload import DownloadMan, parse_manifest
mountpoint = '/mnt/arXiv_src/'
allfiles = 'allfiles.csv'
doun = 'downloaded_log.csv'
error_log = 'error.log'
D = DownloadMan(mountpoint, allfiles, doun, error_log)

In [238]:
import process as P
import re
name = re.match(r'.*([a-z]+)([0-9]{7}).+', '0703/math0703071.gz') 
name.group(1)
P.tar2api2('0703/math0703071.gz', sep='*')
P.detect_format('0701/1701.00112')

(1, '[0-9]{4}/([0-9]{4})\\.([0-9]{4,5})')

In [285]:
import process as P
xx = P.Xtraction('/mnt/arXiv_src/src/arXiv_src_0701_003.tar')

ValueError: There is no file at /mnt/arXiv_src/src/arXiv_src_0701_003.tar

In [284]:
xx.extract_tar('../check6', 'math.AG')

writing file 0701/math0701036               writing file 0701/math0701046               writing file 0701/math0701053               writing file 0701/math0701074               writing file 0701/math0701091               writing file 0701/math0701105               writing file 0701/math0701110               writing file 0701/math0701115               writing file 0701/math0701138               writing file 0701/math0701194               writing file 0701/math0701213               writing file 0701/math0701246               writing file 0701/math0701255               writing file 0701/math0701257               writing file 0701/math0701297               writing file 0701/math0701311               writing file 0701/math0701312               writing file 0701/math0701315               writing file 0701/math0701319               writing file 0701/math0701336               writing file 0701/math0701353               writing file 0701/math0701374               writing file 0701/math0701376   

True

In [192]:
%time xx.extract_tar('../check4', 'math.DG')

writing file 1501/1501.00859.gz               writing file 1501/1501.00930.gz               writing file 1501/1501.00906.gz               writing file 1501/1501.01120.gz               successful extraction of  arXiv_src_1501_002.tar      
CPU times: user 5.45 s, sys: 58 ms, total: 5.51 s
Wall time: 5.73 s


True

In [292]:
[s for s in xx.art_lst if '0701590' in s]

['0701/math0701590.gz']

In [51]:
ll = list(range(13))
length = 4
lal = []
i = 0
while i*length < len(ll):
    seclen = min((i+1)*length, len(ll))
    lal += list(range(i*length, seclen))
    i += 1
print(lal)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [105]:
xx.path_dir('../mathy')

'../mathy/1804_001'

In [99]:
D.check_md5()

NameError: name 'D' is not defined

In [68]:
new_all = parse_manifest('/mnt/arXiv_src/src/arXiv_src_manifest.xml')

In [12]:
fname = 'src/arXiv_src_1607_002.tar'
D.md5sum(fname)

'1b621f41cd46b9d3388351ba9e3aefaf'

In [13]:
D.allfiles_df[D.allfiles_df.filename == fname]

,content_md5sum,filename,first_item,last_item,md5sum,num_items,seq_num,size,timestamp,yymm
1218,8979b2424c6d31c8be791aa675ef05ba,src/arXiv_src_1607_002.tar,1607.00463,1607.00988,a29303e80c4f753e17209a9f6238f2c5,526,2,545216578,2017-07-05 05:41:14,1607


In [14]:
new_all[new_all.filename == fname]

,content_md5sum,filename,first_item,last_item,md5sum,num_items,seq_num,size,timestamp,yymm
1218,2446538d56ddee4f0e28ee65937b6189,src/arXiv_src_1607_002.tar,1607.00463,1607.00988,1b621f41cd46b9d3388351ba9e3aefaf,526,2,556653925,2018-07-05 06:37:36,1607


In [22]:
D.allfiles_df[D.allfiles_df.filename == fname].md5sum.iloc[0]

'5c0c499b3cc7c4961e522aa55e8974a5'

In [24]:
sum([D.filesize(fn[0]) for fn in list(D.downloaded_df.values)])/1e9

116.614211313

In [ ]:
keep_going = 0
while keep_going == 0:
    keep_going = D.get_next()
    if D.check_md5():
        pass
    else:
        break

Dowloand of file src/arXiv_src_0003_001.tar successful
Dowloand of file src/arXiv_src_0004_001.tar successful
Dowloand of file src/arXiv_src_0005_001.tar successful
Dowloand of file src/arXiv_src_0006_001.tar successful
Dowloand of file src/arXiv_src_0007_001.tar successful
Dowloand of file src/arXiv_src_0008_001.tar successful
Dowloand of file src/arXiv_src_0009_001.tar successful
Dowloand of file src/arXiv_src_0010_001.tar successful
Dowloand of file src/arXiv_src_0011_001.tar successful
Dowloand of file src/arXiv_src_0012_001.tar successful
Dowloand of file src/arXiv_src_0101_001.tar successful
Dowloand of file src/arXiv_src_0102_001.tar successful
Dowloand of file src/arXiv_src_0103_001.tar successful
Dowloand of file src/arXiv_src_0104_001.tar successful
Dowloand of file src/arXiv_src_0105_001.tar successful
Dowloand of file src/arXiv_src_0106_001.tar successful
Dowloand of file src/arXiv_src_0107_001.tar successful
Dowloand of file src/arXiv_src_0108_001.tar successful
Dowloand o

In [45]:
import urllib.request as req
url = 'http://export.arxiv.org/api/query?search_query=id:1804.00020&start=0&max_results=5'
data = req.urlopen(url).read()
root = ET.fromstring(data)
res = root.getchildren()[-1]
res.find('title').text

AttributeError: 'NoneType' object has no attribute 'text'

In [38]:
for el in res.getchildren():
    print(el.tag)

{http://www.w3.org/2005/Atom}id
{http://www.w3.org/2005/Atom}updated
{http://www.w3.org/2005/Atom}published
{http://www.w3.org/2005/Atom}title
{http://www.w3.org/2005/Atom}summary
{http://www.w3.org/2005/Atom}author
{http://www.w3.org/2005/Atom}author
{http://arxiv.org/schemas/atom}comment
{http://www.w3.org/2005/Atom}link
{http://www.w3.org/2005/Atom}link
{http://arxiv.org/schemas/atom}primary_category
{http://www.w3.org/2005/Atom}category


In [33]:
import sys
sys.path.insert(0,'arxiv.py/')
import arxiv

In [41]:
import arxiv
D = arxiv.query(id_list=['math/9909175'])
len(D)
#for d in D[0].keys():
#    print(d,D[0][d])

1

In [44]:
D[0]

{'id': 'http://arxiv.org/abs/math/9909175v1',
 'guidislink': True,
 'updated': '1999-09-29T15:08:44Z',
 'updated_parsed': time.struct_time(tm_year=1999, tm_mon=9, tm_mday=29, tm_hour=15, tm_min=8, tm_sec=44, tm_wday=2, tm_yday=272, tm_isdst=0),
 'published': '1999-09-29T15:08:44Z',
 'published_parsed': time.struct_time(tm_year=1999, tm_mon=9, tm_mday=29, tm_hour=15, tm_min=8, tm_sec=44, tm_wday=2, tm_yday=272, tm_isdst=0),
 'title': 'Calabi-Yau threefolds of quotient type',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'http://export.arxiv.org/api/query?search_query=&id_list=math%2F9909175&start=0&max_results=10&sortBy=relevance&sortOrder=descending',
  'value': 'Calabi-Yau threefolds of quotient type'},
 'summary': 'First, we classify Calabi-Yau threefolds with infinite fundamental group by\nmeans of their minimal splitting coverings introduced by Beauville, and deduce\nthat the nef cone is a rational simplicial cone and any rational nef divisor is\nsemi-ample 

In [268]:
D = arxiv.query(id_list=['1601.00103'])
D[0].id

'http://arxiv.org/abs/1601.00103v2'

In [221]:
arxiv.query(id_list=['1805.02773'])[0].links

[{'href': 'http://arxiv.org/abs/1805.02773v1',
  'rel': 'alternate',
  'type': 'text/html'},
 {'title': 'pdf',
  'href': 'http://arxiv.org/pdf/1805.02773v1',
  'rel': 'related',
  'type': 'application/pdf'}]

In [26]:
year_month = '1804.'
id_list = [year_month + str(k).zfill(5) for k in range(1,263)]
dicts = arxiv.query(id_list=id_list, max_results=300)

In [89]:
diff_geom = [d for d in dicts if d.get('tags',None)[0]['term']=='math.DG']
diff_geom

[{'id': 'http://arxiv.org/abs/1804.00018v1',
  'guidislink': True,
  'updated': '2018-03-30T18:11:49Z',
  'updated_parsed': time.struct_time(tm_year=2018, tm_mon=3, tm_mday=30, tm_hour=18, tm_min=11, tm_sec=49, tm_wday=4, tm_yday=89, tm_isdst=0),
  'published': '2018-03-30T18:11:49Z',
  'published_parsed': time.struct_time(tm_year=2018, tm_mon=3, tm_mday=30, tm_hour=18, tm_min=11, tm_sec=49, tm_wday=4, tm_yday=89, tm_isdst=0),
  'title': 'Uniqueness of convex ancient solutions to mean curvature flow in higher\n  dimensions',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'http://export.arxiv.org/api/query?search_query=&id_list=1804.00001%2C1804.00002%2C1804.00003%2C1804.00004%2C1804.00005%2C1804.00006%2C1804.00007%2C1804.00008%2C1804.00009%2C1804.00010%2C1804.00011%2C1804.00012%2C1804.00013%2C1804.00014%2C1804.00015%2C1804.00016%2C1804.00017%2C1804.00018%2C1804.00019%2C1804.00020%2C1804.00021%2C1804.00022%2C1804.00023%2C1804.00024%2C1804.00025%2C1804.00026%2C1

In [90]:
[d.title for d in diff_geom]

['Uniqueness of convex ancient solutions to mean curvature flow in higher\n  dimensions',
 'An almost splitting theorem for a warped product space',
 'Surfaces of revolution with prescribed mean and skew curvatures in\n  Lorentz-Minkowski space']

In [29]:
math = [d for d in dicts if 'math' in d['tags'][0]['term']]

In [43]:
set([d['tags'][0]['term'] for d in math])

{'math-ph',
 'math.AG',
 'math.AP',
 'math.CO',
 'math.CV',
 'math.DG',
 'math.DS',
 'math.FA',
 'math.HO',
 'math.KT',
 'math.LO',
 'math.MG',
 'math.NA',
 'math.NT',
 'math.OA',
 'math.OC',
 'math.PR',
 'math.QA',
 'math.RA',
 'math.RT',
 'math.SP',
 'math.ST'}

In [86]:
import re
def api2tar(id_str):
    '''
    The arxiv API produces ids in the format:
    http://arxiv.org/abs/1804.00018v1',
 http://arxiv.org/abs/1804.00193v1',
 http://arxiv.org/abs/1804.00259v2
    The tar files have name format:
    <TarInfo '1804/1804.00020.gz' at 0x7f0ba68c8cc8>
    '''
    name = re.match(r'.*([0-9]{4})\.([0-9]{5})',id_str)
    return name.group(1) + '/' + name.group(1) + '.' + name.group(2) + '.gz'
    
    
[api2tar(d.id) for d in dicts if d.get('tags',None)[0]['term']=='math.DG']

['1804/1804.00018.gz', '1804/1804.00193.gz', '1804/1804.00259.gz']

In [78]:
DD = dicts[0]
DD

{'id': 'http://arxiv.org/abs/1804.00001v1',
 'guidislink': True,
 'updated': '2018-03-30T11:08:38Z',
 'updated_parsed': time.struct_time(tm_year=2018, tm_mon=3, tm_mday=30, tm_hour=11, tm_min=8, tm_sec=38, tm_wday=4, tm_yday=89, tm_isdst=0),
 'published': '2018-03-30T11:08:38Z',
 'published_parsed': time.struct_time(tm_year=2018, tm_mon=3, tm_mday=30, tm_hour=11, tm_min=8, tm_sec=38, tm_wday=4, tm_yday=89, tm_isdst=0),
 'title': 'Phenomenological analysis of densification mechanism during spark plasma\n  sintering of MgAl2O4',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'http://export.arxiv.org/api/query?search_query=&id_list=1804.00001%2C1804.00002%2C1804.00003%2C1804.00004%2C1804.00005%2C1804.00006%2C1804.00007%2C1804.00008%2C1804.00009%2C1804.00010%2C1804.00011%2C1804.00012%2C1804.00013%2C1804.00014%2C1804.00015%2C1804.00016%2C1804.00017%2C1804.00018%2C1804.00019%2C1804.00020%2C1804.00021%2C1804.00022%2C1804.00023%2C1804.00024%2C1804.00025%2C1804.00026%2C18

In [6]:
import tarfile

with tarfile.open('/home/luis/MisDocumentos/arxivBulkDownload/arXiv_src_1804_001.tar') as ff:
    art_lst = [k for k in ff.getmembers()]

In [80]:
ti = art_lst[1]
ti

<TarInfo '1804/1804.00020.gz' at 0x7f0ba68c8cc8>

In [52]:
ti.get_info()

{'name': '1804/1804.00020.gz',
 'mode': 420,
 'uid': 10519,
 'gid': 10519,
 'size': 15177,
 'mtime': 1522713637,
 'chksum': 5742,
 'type': b'0',
 'linkname': '',
 'uname': 'e-prints',
 'gname': 'e-prints',
 'devmajor': 0,
 'devminor': 0}

In [4]:
import gzip
import tarfile
with tarfile.open('/home/luis/MisDocumentos/arxivBulkDownload/arXiv_src_1503_017.tar') as ff:
    zhu_gz = ff.extractfile('1503/1503.08375.gz')
    zhu = gzip.open(zhu_gz,'rb')
    zhu_str = zhu.read()

FileNotFoundError: [Errno 2] No such file or directory: '/home/luis/MisDocumentos/arxivBulkDownload/arXiv_src_1503_017.tar'

In [95]:
with open('./zhu_untared.tex', 'w') as f:
    f.write(zhu_str.decode('utf-8'))

In [93]:
print(zhu_str)

b"\\documentclass[psamsfonts,a4paper]{amsart}\r\n\r\n%-------Packages---------\r\n\\usepackage{amssymb,amsfonts}\r\n\\usepackage{amsaddr}\r\n\\usepackage[all,arc]{xy}\r\n\\usepackage{enumerate}\r\n%\\usepackage{mathrsfs}\r\n\\usepackage{tikz}\r\n\\usepackage{parskip}\r\n\\usepackage[margin=2.5cm]{geometry}\r\n\\usepackage{eucal}\r\n\r\n%--------Theorem Environments--------\r\n%theoremstyle{plain} --- default\r\n\r\n\r\n\\newtheorem{thm}{Theorem}[section]\r\n\r\n% Reinsert ``[section]'' here if you want theorems numbered by section.\r\n\r\n\\newtheorem{cor}[thm]{Corollary}\r\n\\newtheorem{prop}[thm]{Proposition}\r\n\\newtheorem{lemma}[thm]{Lemma}\r\n\\newtheorem{conj}[thm]{Conjecture}\r\n\\newtheorem{quest}[thm]{Question}\r\n\r\n\\theoremstyle{definition}\r\n\\newtheorem{defn}[thm]{Definition}\r\n\\newtheorem{defns}[thm]{Definitions}\r\n\\newtheorem{con}[thm]{Construction}\r\n\\newtheorem{exmp}[thm]{Example}\r\n\\newtheorem{exmps}[thm]{Examples}\r\n\\newtheorem{notn}[thm]{Notation}\r\n\

In [48]:
import gzip
import chardet
import tarfile
file_tar = tarfile.open('/mnt/arXiv_src/src/arXiv_src_9904_001.tar')
#for f in file_tar.getmembers():
#    print(f.get_info())

tar2 = file_tar.extractfile('9904/math9904086.gz')
#print(chardet.detect(tar2))
#with open('../todo_macizo.pdf','w') as todo_macizo:
#    todo_macizo.write(tar2.read().decode('utf-8'))
#with tarfile.open(fileobj=tar2) as tars:
#    for t in tars:
 #       print(t.get_info()['name'])

#gz = gzip.open(tar2, 'rb')

with gzip.open(tar2,'r') as tars:
    t = tars.read()


#print(t.decode('utf-8'))
chardet.detect(t)

#with open('../holita.txt','w') as holaf:
#    holaf.write(allfile.read().decode('utf-8'))
    

{'encoding': 'ISO-2022-JP', 'confidence': 0.99, 'language': 'Japanese'}

In [50]:
t.decode('cp932')
#t.decode(errors='ignore')

'\\documentclass[12pt]{amsart}\n\\usepackage{amssymb}\n\\setlength{\\headheight}{8pt}\n\\setlength{\\textheight}{22.4cm}\n\\setlength{\\textwidth}{14.5cm}\n\\setlength{\\oddsidemargin}{.1cm}\n\\setlength{\\evensidemargin}{.1cm}\n\\setlength{\\topmargin}{0.2cm}\n\\input{xypic}\n\\begin{document}\n\\theoremstyle{plain}\n\\newtheorem{thm}{Theorem}[section]\n\\newtheorem*{thm1}{Theorem 1}\n\\newtheorem*{thm2}{Theorem 2}\n\\newtheorem{lemma}[thm]{Lemma}\n\\newtheorem{lem}[thm]{Lemma}\n\\newtheorem{cor}[thm]{Corollary}\n\\newtheorem{prop}[thm]{Proposition}\n\\newtheorem{propose}[thm]{Proposition}\n\\newtheorem{variant}[thm]{Variant}\n\\theoremstyle{definition}\n\\newtheorem{notations}[thm]{Notations}\n\\newtheorem{rem}[thm]{Remark}\n\\newtheorem{rmk}[thm]{Remark}\n\\newtheorem{rmks}[thm]{Remarks}\n\\newtheorem{defn}[thm]{Definition}\n\\newtheorem{ex}[thm]{Example}\n\\newtheorem{claim}[thm]{Claim}\n\\newtheorem{ass}[thm]{Assumption}\n\\numberwithin{equation}{section}\n\\newcounter{elno}      

In [348]:
try:
    t.decode()
except UnicodeDecodeError as e:
    print('todo bien')
    print(e)
    

todo bien
'utf-8' codec can't decode byte 0x8f in position 0: invalid start byte


In [306]:
import difflib
Diffs = difflib.unified_diff(t.decode(errors='ignore').splitlines(), t.decode('koi8-r')) 
for d in Diffs:
    print(d)

--- 

+++ 

@@ -1,900 +1,40022 @@

-\documentclass[12pt]{amsart}
-\usepackage{amsmath,amssymb,amsthm}
-%\usepackage[russian]{babel}
-%\usepackage[koi8-r]{inputenc}
-\pagestyle{plain}
-\sloppy
-
-
-\makeatletter
-\newcommand{\symbitem}[1]{\item[#1]%
-\renewcommand{\@currentlabel}{#1}\ignorespaces}
-\makeatother
-
-\renewcommand{\labelenumi}{\theenumi.}
-
-\makeatletter\@addtoreset{equation}{section} \makeatother
-\renewcommand{\theequation}{\thesection.\arabic{equation}}
- 
-
-\newtheorem{theorem}[equation]{Theorem}
-\newtheorem*{theorem*}{Theorem}
-\newtheorem{proposition}[equation]{Proposition}
-\newtheorem{lemma}[equation]{Lemma}
-\newtheorem{corollary}[equation]{Corollary}
-
-\theoremstyle{definition}
-\newtheorem{definition}[equation]{Definition}
-
-\theoremstyle{remark}
-\newtheorem{remark}[equation]{Remark}
-
-
-\newcommand{\tit}{On rationality of nonsingular threefolds\\ 
-with a pencil of Del Pezzo surfaces of degree $4$}
-\title{\tit}
-\author{C.\,Shramov}
-\thanks{The work wa

-${\mathrm{Sing}}(D_1)=A_1\cap A_2\cap Y_3$, the condition \ref{singcond} 
-means that  
-$D_2\cap A_1\cap A_2\cap Y_3=\varnothing$. 
-In particular,  $A_1A_2D_2\vert_{Y_3}=0$ must hold, i.\,e., as 
-$|A_1|=|M_{Y_3}+(d_1+b_1)L_{Y_3}|$, $|A_2|=|M_{Y_3}+(d_2+b_1)L_{Y_3}|$, and 
-$Y_3\cong{\mathbb{F}}(d_3, d_4, 0)$, the latter condition may be rewritten as
-$2(d_1+d_2+d_3+d_4)+4b_1+b_2=0$. 
-
-If $d_1+d_4+b_1\geqslant 0$, then $A_1$ and $A_2$ have no common components, 
-and their intersection is an effective curve on $Y_3$, 
-hence if $Y_5\not\subset Bs|D_2|$ the condition $A_1A_2D_2\vert_{Y_3}=0$ 
-implies $D_2\cap A_1\cap A_2\cap Y_3=\varnothing$. 
-If $d_1+b_1\geqslant 0$, then $A_1$ and $A_2$ have no common components, 
-and $Y_5\not\subset A_1$, so that if $Y_4\not\subset Bs|D_2|$ the condition
-$A_1A_2D_2\vert_{Y_3}=0$ implies $D_2\cap A_1\cap A_2\cap Y_3=\varnothing$.
-\end{proof}
-
-\section{Proof of Theorem~\ref{maintheorem}}\label{proofofmaintheorem}
-
-\subsection{Case $Bs|D_1

+s
+t
+r
+a
+c
+t
+}
+

+W
+e
+ 
+p
+r
+o
+v
+e
+ 
+a
+ 
+c
+r
+i
+t
+e
+r
+i
+o
+n
+ 
+o
+f
+ 
+n
+o
+n
+s
+i
+n
+g
+u
+l
+a
+r
+i
+t
+y
+ 
+o
+f
+ 
+a
+ 
+c
+o
+m
+p
+l
+e
+t
+e
+ 
+i
+n
+t
+e
+r
+s
+e
+c
+t
+i
+o
+n
+ 
+o
+f
+ 
+t
+w
+o
+ 
+f
+i
+b
+e
+r
+w
+i
+s
+e
+

+q
+u
+a
+d
+r
+i
+c
+s
+ 
+i
+n
+ 
+

+$
+{
+\
+m
+a
+t
+h
+b
+b
+{
+P
+}
+}
+_
+{
+{
+\
+m
+a
+t
+h
+b
+b
+{
+P
+}
+}
+^
+1
+}
+(
+{
+\
+m
+a
+t
+h
+c
+a
+l
+{
+O
+}
+}
+(
+d
+_
+1
+)
+\
+o
+p
+l
+u
+s
+\
+l
+d
+o
+t
+s
+\
+o
+p
+l
+u
+s
+{
+\
+m
+a
+t
+h
+c
+a
+l
+{
+O
+}
+}
+(
+d
+_
+5
+)
+)
+$
+.
+ 
+

+A
+s
+ 
+a
+ 
+c
+o
+r
+o
+l
+l
+a
+r
+y
+ 
+w
+e
+ 
+d
+e
+r
+i
+v
+e
+ 
+t
+h
+e
+ 
+f
+o
+l
+l
+o
+w
+i
+n
+g
+ 
+a
+d
+d
+i
+t
+i
+o
+n
+ 
+t
+o
+ 
+t
+h
+e
+ 
+A
+l
+e
+x
+e
+e
+v
+ 
+t
+h
+e
+o
+r
+e
+m
+ 
+o
+n
+ 
+

+r
+a
+t
+i
+o
+n
+a
+l
+i
+t
+y
+ 
+o
+f
+ 
+s
+t
+a
+n
+d
+a
+r
+d
+ 
+D
+e
+l
+ 
+P
+e
+z
+z
+o
+ 
+f
+i
+b
+r
+a
+t
+i
+o
+n
+s
+ 
+o
+f
+ 
+d
+e
+g
+r
+e
+e
+ 
+$
+4
+$
+ 
+o
+v
+e
+r
+ 
+

+r
+s
+e
+c
+t
+i
+o
+n
+ 
+o
+f
+ 
+t
+w
+o
+ 
+g
+e
+n
+e
+r
+a
+l
+ 
+m
+e
+m
+b
+e
+r
+s
+

+o
+f
+ 
+g
+i
+v
+e
+n
+ 
+l
+i
+n
+e
+a
+r
+ 
+s
+y
+s
+t
+e
+m
+s
+ 
+$
+|
+D
+_
+1
+|
+$
+ 
+a
+n
+d
+ 
+$
+|
+D
+_
+2
+|
+$
+ 
+o
+f
+ 
+f
+i
+b
+e
+r
+w
+i
+s
+e
+ 
+q
+u
+a
+d
+r
+i
+c
+s
+ 
+o
+n
+ 
+$
+Y
+$
+ 
+

+i
+s
+ 
+n
+o
+n
+s
+i
+n
+g
+u
+l
+a
+r
+ 
+(
+o
+r
+,
+ 
+e
+q
+u
+i
+v
+a
+l
+e
+n
+t
+l
+y
+,
+ 
+i
+f
+ 
+t
+h
+e
+r
+e
+ 
+e
+x
+i
+s
+t
+ 
+s
+u
+c
+h
+ 
+d
+i
+v
+i
+s
+o
+r
+s
+ 
+i
+n
+ 
+t
+h
+e
+s
+e
+ 
+

+l
+i
+n
+e
+a
+r
+ 
+s
+y
+s
+t
+e
+m
+s
+ 
+t
+h
+a
+t
+ 
+t
+h
+e
+i
+r
+ 
+i
+n
+t
+e
+r
+s
+e
+c
+t
+i
+o
+n
+ 
+i
+s
+ 
+n
+o
+n
+s
+i
+n
+g
+u
+l
+a
+r
+)
+.
+

+

+L
+e
+t
+ 
+u
+s
+ 
+f
+i
+x
+ 
+s
+o
+m
+e
+ 
+n
+o
+t
+a
+t
+i
+o
+n
+s
+ 
+t
+o
+ 
+f
+o
+r
+m
+u
+l
+a
+t
+e
+ 
+t
+h
+e
+ 
+a
+n
+s
+w
+e
+r
+ 
+t
+o
+ 
+t
+h
+e
+ 
+l
+a
+t
+t
+e
+r
+ 
+q
+u
+e
+s
+t
+i
+o
+n
+.
+

+L
+e
+t
+ 
+$
+Y
+=
+{
+\
+m
+a
+t
+h
+b
+b
+{
+F
+}
+}
+(
+d
+_
+1
+

+2
++
+b
+_
+1
+<
+0
+$
+,
+ 
+

+$
+d
+_
+3
++
+d
+_
+4
++
+b
+_
+1
+<
+0
+$
+,
+ 
+$
+b
+_
+2
+\
+g
+e
+q
+s
+l
+a
+n
+t
+ 
+0
+$
+.
+

+%
+(
+6
+,
+ 
+4
+,
+ 
+3
+,
+ 
+2
+,
+ 
+0
+)
+,
+ 
+-
+6
+,
+ 
+0
+

+\
+i
+t
+e
+m
+%
+\
+s
+y
+m
+b
+i
+t
+e
+m
+{
+е
+}
+\
+!
+\
+t
+e
+x
+t
+u
+p
+{
+)
+}
+

+\
+l
+a
+b
+e
+l
+{
+y
+4
+c
+a
+s
+e
+6
+}
+ 
+$
+d
+_
+1
+>
+d
+_
+2
+$
+,
+ 
+$
+d
+_
+1
+>
+d
+_
+3
++
+d
+_
+4
+$
+,
+ 
+$
+d
+_
+2
++
+d
+_
+4
+>
+d
+_
+1
+$
+,
+

+$
+b
+_
+1
+=
+-
+d
+_
+1
+$
+,
+ 
+$
+b
+_
+2
+<
+0
+$
+,
+ 
+$
+2
+d
+_
+4
++
+b
+_
+2
+\
+g
+e
+q
+s
+l
+a
+n
+t
+ 
+0
+$
+,
+ 
+$
+d
+_
+3
++
+b
+_
+2
+\
+g
+e
+q
+s
+l
+a
+n
+t
+ 
+0
+$
+ 
+o
+r
+ 
+

+$
+ 
+d
+_
+2
++
+b
+_
+2
+=
+0
+$
+.
+

+%
+(
+8
+,
+ 
+6
+,
+ 
+4
+,
+ 
+3
+,
+ 
+0
+)
+,
+ 
+-
+8
+,
+ 
+-
+6
+

+%
+(
+8
+,
+ 
+6
+,
+ 
+4
+,
+ 
+3
+,
+ 
+0
+)
+,
+ 
+-
+8
+,
+ 
+-
+4
+

+\
+i
+t
+e
+m
+%
+\
+s
+y
+m
+b
+i
+t
+e
+m
+{
+ж
+}
+\
+!
+\
+t
+e
+x
+t
+u
+p
+{
+)
+}
+

+\
+l
+a
+b
+e
+l
+{
+y
+4
+c
+a
+

...

+~
+\
+r
+e
+f
+{
+m
+a
+i
+n
+t
+h
+e
+o
+r
+e
+m
+}
+)
+.
+ 
+

+

+\
+s
+u
+b
+s
+u
+b
+s
+e
+c
+t
+i
+o
+n
+{
+C
+a
+s
+e
+ 
+$
+d
+_
+1
++
+b
+_
+1
+\
+g
+e
+q
+s
+l
+a
+n
+t
+ 
+0
+$
+,
+ 
+$
+d
+_
+2
++
+d
+_
+4
++
+b
+_
+1
+\
+g
+e
+q
+s
+l
+a
+n
+t
+ 
+0
+$
+,
+ 
+

+$
+d
+_
+2
++
+b
+_
+1
+<
+0
+$
+,
+ 
+$
+d
+_
+3
++
+d
+_
+4
++
+b
+_
+1
+<
+0
+$
+.
+}
+

+W
+e
+ 
+h
+a
+v
+e
+ 
+$
+\
+a
+l
+p
+h
+a
+_
+{
+2
+5
+}
+=
+\
+a
+l
+p
+h
+a
+_
+{
+3
+4
+}
+=
+\
+a
+l
+p
+h
+a
+_
+{
+3
+5
+}
+=
+0
+$
+,
+ 
+a
+n
+d
+ 
+$
+D
+_
+1
+$
+ 
+h
+a
+s
+ 
+a
+t
+ 
+m
+o
+s
+t
+ 
+i
+s
+o
+l
+a
+t
+e
+d
+ 
+

+s
+i
+n
+g
+u
+l
+a
+r
+i
+t
+i
+e
+s
+.
+ 
+I
+f
+ 
+$
+d
+_
+1
++
+b
+_
+1
+>
+0
+$
+,
+ 
+t
+h
+e
+n
+ 
+$
+D
+_
+1
+$
+ 
+h
+a
+s
+ 
+s
+o
+m
+e
+ 
+s
+i
+n
+g
+u
+l
+a
+r
+ 
+p
+o
+i
+n
+t
+s
+ 
+

+o
+n
+ 
+$
+Y
+_
+5
+$
+,
+ 
+h
+e
+n
+c
+e
+ 
+\
+r
+e
+f
+{
+s
+i
+n
+g
+c
+o
+n
+d
+}
+ 
+i
+m
+p
+l
+i
+e
+s
+ 
+$
+b
+_
+2
+\
+g
+e
+q
+s
+l
+a
+n
+t
+ 
+0
+$
+.
+

+T
+h
+e
+ 
+

+p
+h
+a
+_
+{
+2
+3
+}
+x
+_
+3
++
+\
+a
+l
+p
+h
+a
+_
+{
+2
+4
+}
+x
+_
+4
++
+\
+a
+l
+p
+h
+a
+_
+{
+2
+5
+}
+x
+_
+5
+$
+a
+r
+e
+ 
+z
+e
+r
+o
+ 
+o
+n
+ 
+$
+Y
+_
+3
+$
+.
+

+

+L
+e
+t
+ 
+u
+s
+ 
+c
+o
+n
+s
+i
+d
+e
+r
+ 
+s
+e
+v
+e
+r
+a
+l
+ 
+p
+o
+s
+s
+i
+b
+i
+l
+i
+t
+i
+e
+s
+.
+

+

+\
+s
+u
+b
+s
+u
+b
+s
+e
+c
+t
+i
+o
+n
+{
+C
+a
+s
+e
+ 
+$
+d
+_
+2
++
+d
+_
+3
++
+b
+_
+1
+<
+0
+$
+.
+}
+ 
+

+W
+e
+ 
+h
+a
+v
+e
+ 
+$
+\
+a
+l
+p
+h
+a
+_
+{
+2
+3
+}
+=
+\
+a
+l
+p
+h
+a
+_
+{
+2
+4
+}
+=
+\
+a
+l
+p
+h
+a
+_
+{
+2
+5
+}
+=
+0
+$
+,
+

+a
+n
+d
+ 
+$
+{
+\
+m
+a
+t
+h
+r
+m
+{
+S
+i
+n
+g
+}
+}
+(
+D
+_
+1
+)
+$
+ 
+c
+o
+n
+t
+a
+i
+n
+s
+ 
+a
+ 
+l
+i
+n
+e
+ 
+i
+n
+ 
+a
+ 
+g
+e
+n
+e
+r
+a
+l
+ 
+f
+i
+b
+e
+r
+ 
+o
+f
+ 
+$
+\
+v
+a
+r
+p
+h
+i
+$
+,
+ 
+

+h
+e
+n
+c
+e
+ 
+\
+r
+e
+f
+{
+s
+i
+n
+g
+c
+o
+n
+d
+}
+ 
+d
+o
+e
+s
+n
+'
+t
+ 
+h
+o
+l
+d
+.
+

+

+\
+s
+u
+b
+s
+u
+b
+s
+e
+c
+t
+i
+o
+n
+{
+C
+a
+s
+e
+ 
+$
+d
+_
+2
++
+d
+_
+3
++
+b
+

+_
+3
++
+b
+_
+2
+\
+g
+e
+q
+s
+l
+a
+n
+t
+ 
+0
+$
+.
+ 
+

+B
+y
+ 
+L
+e
+m
+m
+a
+~
+\
+r
+e
+f
+{
+l
+e
+m
+m
+a
+:
+Y
+3
+-
+s
+i
+n
+g
+}
+ 
+t
+h
+e
+ 
+c
+o
+n
+d
+i
+t
+i
+o
+n
+ 
+\
+r
+e
+f
+{
+s
+i
+n
+g
+c
+o
+n
+d
+}
+ 
+i
+m
+p
+l
+i
+e
+s
+ 
+

+$
+$
+0
+=
+2
+(
+d
+_
+1
++
+d
+_
+2
++
+d
+_
+3
++
+d
+_
+4
+)
++
+4
+b
+_
+1
++
+b
+_
+2
+=
+2
+(
+d
+_
+1
++
+b
+_
+1
+)
++
+2
+(
+d
+_
+2
++
+d
+_
+4
++
+b
+_
+1
+)
++
+2
+d
+_
+3
++
+b
+_
+2
+,
+$
+$
+

+i
+.
+\
+,
+e
+.
+ 
+$
+d
+_
+2
++
+d
+_
+4
++
+b
+_
+1
+=
+d
+_
+1
++
+b
+_
+1
+=
+2
+d
+_
+3
++
+b
+_
+2
+=
+0
+$
+.
+ 
+I
+f
+ 
+u
+n
+d
+e
+r
+ 
+t
+h
+e
+s
+e
+ 
+a
+s
+s
+u
+m
+p
+t
+i
+o
+n
+s
+ 
+$
+d
+_
+3
+>
+d
+_
+4
+$
+,
+ 
+

+t
+h
+e
+n
+ 
+$
+B
+s
+|
+D
+_
+2
+|
+=
+Y
+_
+4
+$
+,
+ 
+a
+n
+d
+ 
+b
+y
+ 
+L
+e
+m
+m
+a
+~
+\
+r
+e
+f
+{
+l
+e
+m
+m
+a
+:
+M
+4
+-
+m
+i
+n
+o
+r
+s
+}
+ 
+t
+h
+e
+ 
+c
+o
+n
+d
+i
+t
+i
+o
+n
+ 
+

+\
+r
+e
+f
+{
+c
+o
+m
+m
+o
+n
+b
+s
+c
+o
+n
+d
+}
+ 
+i
+m
+p
+l
+i
+e
+

+e
+s
+

+i
+n
+ 
+t
+h
+e
+ 
+f
+i
+b
+e
+r
+s
+ 
+o
+f
+ 
+$
+\
+v
+a
+r
+p
+h
+i
+$
+,
+ 
+p
+a
+s
+s
+i
+n
+g
+ 
+t
+h
+r
+o
+u
+g
+h
+ 
+$
+Y
+_
+5
+$
+.
+ 
+E
+a
+c
+h
+ 
+l
+i
+n
+e
+ 
+$
+l
+$
+ 
+w
+i
+t
+h
+ 
+t
+h
+i
+s
+ 
+

+p
+r
+o
+p
+e
+r
+t
+y
+,
+ 
+w
+r
+i
+t
+t
+e
+n
+ 
+d
+o
+w
+n
+ 
+i
+n
+ 
+t
+h
+e
+ 
+c
+o
+o
+r
+d
+i
+n
+a
+t
+e
+s
+ 
+ 
+

+$
+(
+x
+_
+1
+,
+ 
+\
+l
+d
+o
+t
+s
+,
+ 
+x
+_
+4
+)
+$
+ 
+i
+n
+ 
+t
+h
+e
+ 
+a
+f
+f
+i
+n
+e
+ 
+s
+p
+a
+c
+e
+ 
+$
+x
+_
+5
+=
+1
+$
+,
+ 
+i
+s
+ 
+g
+i
+v
+e
+n
+ 
+p
+a
+r
+a
+m
+e
+t
+r
+i
+c
+a
+l
+l
+y
+

+b
+y
+ 
+$
+(
+x
+_
+1
+,
+ 
+\
+l
+d
+o
+t
+s
+,
+ 
+x
+_
+4
+)
+=
+s
+(
+v
+_
+1
+,
+ 
+v
+_
+2
+,
+ 
+v
+_
+3
+,
+ 
+v
+_
+4
+)
+$
+.
+ 
+

+T
+h
+e
+ 
+c
+o
+n
+d
+i
+t
+i
+o
+n
+ 
+$
+l
+\
+s
+u
+b
+s
+e
+t
+ 
+D
+_
+1
+\
+c
+a
+p
+ 
+D
+_
+2
+$
+ 
+m
+e
+a
+n
+s
+ 
+t
+h
+a
+t
+ 
+f
+o
+r
+ 
+a
+n
+y
+ 
+$
+s
+$
+ 
+w
+e
+ 
+h
+a
+v
+e
+

+\
+b
+e
+g
+i
+n
+{
+m
+u
+l
+t
+l
+i
+n
+e
+*
+}
+

+(
+(
+

+3
+ 
+(
+1
+9
+9
+7
+)
+,
+ 
+5
+-
+-
+1
+5
+9
+.
+

+\
+e
+n
+d
+{
+d
+o
+c
+u
+m
+e
+n
+t
+}
+

+

+

+



In [245]:
tarfile.ReadError()

tarfile.ReadError()

In [228]:
with tarfile.open('/mnt/arXiv_src/src/arXiv_src_0701_001.tar') as morgan:
    for mor in morgan:
        print(mor.name)


0701
0701/astro-ph0701001.gz
0701/astro-ph0701002.gz
0701/astro-ph0701003.gz
0701/astro-ph0701004.gz
0701/astro-ph0701005.gz
0701/astro-ph0701006.pdf
0701/astro-ph0701007.gz
0701/astro-ph0701008.gz
0701/astro-ph0701009.gz
0701/astro-ph0701010.gz
0701/astro-ph0701011.gz
0701/astro-ph0701012.gz
0701/astro-ph0701013.gz
0701/astro-ph0701014.gz
0701/astro-ph0701015.gz
0701/astro-ph0701016.pdf
0701/astro-ph0701017.gz
0701/astro-ph0701018.gz
0701/astro-ph0701019.gz
0701/astro-ph0701020.gz
0701/astro-ph0701021.gz
0701/astro-ph0701022.gz
0701/astro-ph0701023.gz
0701/astro-ph0701024.gz
0701/astro-ph0701025.gz
0701/astro-ph0701026.gz
0701/astro-ph0701027.gz
0701/astro-ph0701028.gz
0701/astro-ph0701029.gz
0701/astro-ph0701030.gz
0701/astro-ph0701031.gz
0701/astro-ph0701032.gz
0701/astro-ph0701033.gz
0701/astro-ph0701034.gz
0701/astro-ph0701035.gz
0701/astro-ph0701036.pdf
0701/astro-ph0701037.gz
0701/astro-ph0701038.gz
0701/astro-ph0701039.gz
0701/astro-ph0701040.gz
0701/astro-ph0701041.gz
0701/ast

0701/astro-ph0701612.gz
0701/astro-ph0701613.gz
0701/astro-ph0701614.gz
0701/astro-ph0701615.gz
0701/astro-ph0701616.gz
0701/astro-ph0701617.gz
0701/astro-ph0701618.gz
0701/astro-ph0701619.gz
0701/astro-ph0701620.gz
0701/astro-ph0701621.gz
0701/astro-ph0701622.gz
0701/astro-ph0701623.gz
0701/astro-ph0701624.gz
0701/astro-ph0701625.gz
0701/astro-ph0701626.gz
0701/astro-ph0701627.pdf
0701/astro-ph0701628.gz
0701/astro-ph0701629.gz
0701/astro-ph0701630.gz
0701/astro-ph0701631.gz
0701/astro-ph0701632.gz
0701/astro-ph0701633.gz
0701/astro-ph0701634.gz
0701/astro-ph0701635.gz
0701/astro-ph0701636.gz
0701/astro-ph0701637.gz
0701/astro-ph0701638.gz
0701/astro-ph0701639.gz
0701/astro-ph0701640.pdf
0701/astro-ph0701641.pdf
0701/astro-ph0701642.gz
0701/astro-ph0701643.gz
0701/astro-ph0701644.gz
0701/astro-ph0701645.gz
0701/astro-ph0701646.gz
0701/astro-ph0701647.gz
0701/astro-ph0701648.gz
0701/astro-ph0701649.gz
0701/astro-ph0701650.gz
0701/astro-ph0701651.gz
0701/astro-ph0701652.gz
0701/astro-ph

0701/cond-mat0701579.gz
0701/cond-mat0701580.gz
0701/cond-mat0701581.gz
0701/cond-mat0701582.gz
0701/cond-mat0701583.gz
0701/cond-mat0701584.gz
0701/cond-mat0701585.gz
0701/cond-mat0701586.gz
0701/cond-mat0701587.pdf
0701/cond-mat0701588.gz
0701/cond-mat0701589.pdf
0701/cond-mat0701590.gz
0701/cond-mat0701591.gz
0701/cond-mat0701592.gz
0701/cond-mat0701593.gz
0701/cond-mat0701594.gz
0701/cond-mat0701595.gz
0701/cond-mat0701596.gz
0701/cond-mat0701597.gz
0701/cond-mat0701598.gz
0701/cond-mat0701599.pdf
0701/cond-mat0701600.gz
0701/cond-mat0701601.gz
0701/cond-mat0701602.pdf
0701/cond-mat0701603.pdf
0701/cond-mat0701604.gz
0701/cond-mat0701605.gz
0701/cond-mat0701606.gz
0701/cond-mat0701607.gz
0701/cond-mat0701608.gz
0701/cond-mat0701609.gz
0701/cond-mat0701610.gz
0701/cond-mat0701611.gz
0701/cond-mat0701612.gz
0701/cond-mat0701613.gz
0701/cond-mat0701614.gz
0701/cond-mat0701615.gz
0701/cond-mat0701616.gz
0701/cond-mat0701617.pdf
0701/cond-mat0701618.pdf
0701/cond-mat0701619.gz
0701/cond

In [208]:
file_tar.extract?

In [109]:
tarfile.TarFile.extractall?

In [157]:
import chardet
chardet.detect(t)

{'encoding': 'Windows-1252', 'confidence': 0.7292205811322673, 'language': ''}

In [222]:
import glob
glob.glob('/mnt/arXiv_src/src/arXiv_src_1805_*')

['/mnt/arXiv_src/src/arXiv_src_1805_001.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_003.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_004.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_005.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_006.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_007.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_008.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_009.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_010.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_011.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_012.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_013.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_014.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_015.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_016.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_017.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_019.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_020.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_021.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_022.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_023.tar',
 '/mnt/arXiv_src/src/arXiv_src_180

In [172]:
bytes.decode?

In [171]:
type(t)

bytes

In [175]:
import shutil
shutil.rmtree('check_test/')

In [220]:
with open('../check2/1601.00103/1601.00103.tex','r') as fu:
    print(fu.readlines()[:10])

['\\documentclass[12pt]{amsart}\n', '\\usepackage{tikz}\n', '\\usepackage{amssymb}\n', '\\overfullrule=10pt\n', '\\parskip=0pt\n', '\\raggedbottom\n', '\\allowdisplaybreaks\n', '\n', '\\def\\Wnodal{W_{\\scriptstyle\\rm\\!nodal}}\n', '\\def\\P{\\mathbb{P}}\n']


In [286]:
os.listdir('../check6')

['math.0701750',
 'math.0701670',
 'math.0701246',
 'math.0701046',
 'math.0701036',
 'math.0701409',
 'math.0701532',
 'math.0701662',
 'math.0701002',
 'math.0701374',
 'math.0701091',
 'math.0701559',
 'math.0701566',
 'math.0701105',
 'math.0701903',
 'math.0701213',
 'math.0701319',
 'math.0701734',
 'math.0701669',
 'math.0701507',
 'math.0701671',
 'math.0701402',
 'math.0701538',
 'math.0701683',
 'math.0701782',
 'math.0701885',
 'math.0701475',
 'math.0701388',
 'math.0701895',
 'math.0701590',
 'math.0701823',
 'math.0701459',
 'math.0701667',
 'math.0701874',
 'math.0701641',
 'math.0701257',
 'math.0701663',
 'math.0701053',
 'math.0701680',
 'math.0701336',
 'math.0701889',
 'math.0701511',
 'math.0701074',
 'math.0701312',
 'math.0701522',
 'math.0701115',
 'math.0701720',
 'math.0701407',
 'math.0701839',
 'math.0701194',
 'math.0701472',
 'math.0701466',
 'math.0701936',
 'math.0701894',
 'math.0701479',
 'math.0701138',
 'math.0701672',
 'math.0701255',
 'math.0701596

In [287]:
set( ['commentary.txt', 'definitions.sty', 'hyperplane.bbl', 'hyperplane.tex'])== set(['commentary.txt', 'hyperplane.tex', 'definitions.sty', 'hyperplane.bbl'])

True